In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
            
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
 

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('sb_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/kolkata-to-midnapore: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7CCFDEEA2+31554]
	(No symbol) [0x00007FF7CCF57ED9]
	(No symbol) [0x00007FF7CCE1872A]
	(No symbol) [0x00007FF7CCE68434]
	(No symbol) [0x00007FF7CCE6853C]
	(No symbol) [0x00007FF7CCEAF6A7]
	(No symbol) [0x00007FF7CCE8D06F]
	(No symbol) [0x00007FF7CCEAC977]
	(No symbol) [0x00007FF7CCE8CDD3]
	(No symbol) [0x00007FF7CCE5A33B]
	(No symbol) [0x00007FF7CCE5AED1]
	GetHandleVerifier [0x00007FF7CD2E8B1D+3217341]
	GetHandleVerifier [0x00007FF7CD335AE3+3532675]
	GetHandleVerifier [0x00007FF7CD32B0E0+3489152]
	GetHandleVerifier [0x00007FF7CD08E776+750614]
	(No symbol) [0x00007FF7CCF6375F]
	(No symbol) [0x00007FF7CCF5EB14]
	(No symbol) [0x00007FF7CCF5ECA2]
	(No symbol) [0x00007FF7CCF4E16F]
	BaseThreadInitThunk [0x00007FFE5633257D+29]
	RtlUserThreadStart [0x00007FFE572CAF28+40]

Error occurred while scraping bus details for https://www.redbus.in/bus-t

NameError: name 'driver' is not defined

In [2]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...,SBSTC-BURDWAN - KARUNAMOYEE - A/C - 15:45 (BWN...,AC Seater (2+2),15:45,02h 30m,18:15,4.4,INR 200,2 Seats available
1,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...,SBSTC-BURDWAN - KARUNAMOYEE - 16:10 (BWN DEPOT...,Non AC Seater (2+3),16:10,02h 30m,18:40,4.3,INR 100,10 Seats available
2,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...,SBSTC-BURDWAN - KARUNAMOYEE - 16:45 (BWN DEPOT...,Non AC Seater (2+3),16:45,02h 30m,19:15,4.0,INR 100,33 Seats available
3,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...,SBSTC-BURDWAN - KOLKATA - 17:00 (BWN DEPOT) - 41,Non AC Seater (2+3),17:00,02h 00m,19:00,4.1,INR 95,30 Seats available
4,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...,SBSTC-BURDWAN - KARUNAMOYEE - 17:30 (BWN DEPOT...,Non AC Seater (2+3),17:30,02h 30m,20:00,4.1,INR 100,33 Seats available
...,...,...,...,...,...,...,...,...,...,...
415,Siliguri to Malda,https://www.redbus.in/bus-tickets/siliguri-to-...,Shyamoli Paribahan Pvt Ltd,Volvo Multi-Axle A/C Seater/Sleeper (2+1),18:30,05h 40m,00:10,3.1,INR 1200,40 Seats available
416,Siliguri to Malda,https://www.redbus.in/bus-tickets/siliguri-to-...,EASY RIDE,Volvo 9600 Multi-Axle A/C Sleeper (2+1),18:45,05h 10m,23:55,4.7,INR 1500,6 Seats available
417,Siliguri to Malda,https://www.redbus.in/bus-tickets/siliguri-to-...,Shyamoli Paribahan Pvt Ltd,Volvo 9600 A/C Seater/Sleeper (2+1),19:30,05h 10m,00:40,4.5,INR 1200,34 Seats available
418,Kolkata to Suri,https://www.redbus.in/bus-tickets/kolkata-to-suri,SBSTC-KOLKATA - SURI - 16: 30 - (Suri Depot) -...,Non AC Seater (2+3),16:30,05h 00m,21:30,3.4,INR 190,52 Seats available
